## Libraries

In [50]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os



## Contact

In [11]:
server = 'ALI_HANY'
database = 'Care_Stat'

In [12]:
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
)

try:
    conn = pyodbc.connect(conn_str)
    print("✅ Successful connection to the database!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    exit()

✅ Successful connection to the database!


## Get the names of all tables in the database

In [13]:
query_tables = """
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE'
"""

try:
    tables_df = pd.read_sql(query_tables, conn)
    table_names = tables_df['TABLE_NAME'].tolist()
    print(f"✅ Found {len(table_names)} tables: {table_names}")
except Exception as e:
    print(f"❌ Error fetching table names: {e}")
    conn.close()
    exit()

✅ Found 14 tables: ['PatientChronicDiseases', 'Doctors', 'DoctorPhones', 'Departments', 'DoctorDepartment', 'Department_Equipment', 'Patients', 'PatientPhones', 'Visits', 'DoctorWorkplaces', 'Appointments', 'ChronicDiseases', 'Medical_Records', 'Payments']


C:\Users\aliha\AppData\Local\Temp\ipykernel_14732\1046294067.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql(query_tables, conn)


## Load each table as a DataFrame and save it in a dictionary.

In [14]:
all_data = {}

print("\n🔄 Loading all tables into DataFrames...\n")

for table in table_names:
    try:
        query = f"SELECT TOP 10000 * FROM [{table}]"  # نستخدم [] لتفادي الأسماء الخاصة
        all_data[table] = pd.read_sql(query, conn)
        print(f"✅ Loaded '{table}' → {len(all_data[table])} rows, {all_data[table].shape[1]} columns")
    except Exception as e:
        print(f"❌ Failed to load '{table}': {e}")

# Close the connection
conn.close()
print("\n✅ Connection closed.")


🔄 Loading all tables into DataFrames...

✅ Loaded 'PatientChronicDiseases' → 0 rows, 3 columns


C:\Users\aliha\AppData\Local\Temp\ipykernel_14732\1281313748.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_data[table] = pd.read_sql(query, conn)
C:\Users\aliha\AppData\Local\Temp\ipykernel_14732\1281313748.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_data[table] = pd.read_sql(query, conn)


✅ Loaded 'Doctors' → 10000 rows, 14 columns
✅ Loaded 'DoctorPhones' → 10000 rows, 2 columns
✅ Loaded 'Departments' → 10000 rows, 9 columns
✅ Loaded 'DoctorDepartment' → 10000 rows, 3 columns
✅ Loaded 'Department_Equipment' → 10000 rows, 2 columns
✅ Loaded 'Patients' → 10000 rows, 10 columns
✅ Loaded 'PatientPhones' → 10000 rows, 2 columns
✅ Loaded 'Visits' → 10000 rows, 3 columns
✅ Loaded 'DoctorWorkplaces' → 10000 rows, 2 columns
✅ Loaded 'Appointments' → 10000 rows, 5 columns
✅ Loaded 'ChronicDiseases' → 15 rows, 2 columns
✅ Loaded 'Medical_Records' → 10000 rows, 8 columns
✅ Loaded 'Payments' → 10000 rows, 10 columns

✅ Connection closed.


In [16]:
print("📊 Shape of Each Table in 'Care_Stat':\n")
print(f"{'Table Name':<20} {'Rows':<8} {'Columns':<8}")
print("-" * 40)

for table_name, df in all_data.items():
    rows, cols = df.shape
    print(f"{table_name:<20} {rows:<8} {cols:<8}")

📊 Shape of Each Table in 'Care_Stat':

Table Name           Rows     Columns 
----------------------------------------
PatientChronicDiseases 0        3       
Doctors              10000    14      
DoctorPhones         10000    2       
Departments          10000    9       
DoctorDepartment     10000    3       
Department_Equipment 10000    2       
Patients             10000    10      
PatientPhones        10000    2       
Visits               10000    3       
DoctorWorkplaces     10000    2       
Appointments         10000    5       
ChronicDiseases      15       2       
Medical_Records      10000    8       
Payments             10000    10      


## Edit tables


In [ ]:

patients_df = all_data['Patients']  
patient_ids = patients_df['patient_id'].dropna().unique()
print(f"✅ Found {len(patient_ids)} unique patient IDs")


disease_names = [
    'Diabetes', 'Hypertension', 'Asthma', 'Cancer', 'Heart Disease',
    'Stroke', 'Kidney Disease', 'Alzheimer', 'Arthritis', 'Obesity',
    'Epilepsy', 'Glaucoma', 'Thyroid Disease', 'COPD', 'Hepatitis'
]
print(f"📋 Chronic diseases list ({len(disease_names)}): {disease_names}")

# === Generate 10,000 random rows===
target_size = 10000

# Random selection with replay
selected_patient_ids = np.random.choice(patient_ids, size=target_size, replace=True)
selected_disease_names = np.random.choice(disease_names, size=target_size, replace=True)

# === Create the new table===
expanded_df = pd.DataFrame({
    'disease_id': selected_patient_ids,        # ← patient_id as disease_id
    'disease_name': selected_disease_names     # ← Random disease name
})

print(f"✅ Generated {len(expanded_df)} records")
print(f"📊 Shape: {expanded_df.shape}")
print("\n🔍 Sample of generated data:")
print(expanded_df.sample(10).to_string(index=False))

# === 5Save to all_data and CSV. ===
all_data['ChronicDiseases'] = expanded_df

expanded_df.to_csv('ChronicDiseases_10k_linked_to_patients.csv', index=False)
print("📁 Data saved to 'ChronicDiseases_10k_linked_to_patients.csv'")

✅ Found 10000 unique patient IDs
📋 Chronic diseases list (15): ['Diabetes', 'Hypertension', 'Asthma', 'Cancer', 'Heart Disease', 'Stroke', 'Kidney Disease', 'Alzheimer', 'Arthritis', 'Obesity', 'Epilepsy', 'Glaucoma', 'Thyroid Disease', 'COPD', 'Hepatitis']
✅ Generated 10000 records
📊 Shape: (10000, 2)

🔍 Sample of generated data:
 disease_id    disease_name
      39280       Hepatitis
      71149        Diabetes
      33308       Alzheimer
      71752        Epilepsy
       9562       Alzheimer
      67853 Thyroid Disease
      80626 Thyroid Disease
      14065          Stroke
      90139 Thyroid Disease
      55702 Thyroid Disease
📁 Data saved to 'ChronicDiseases_10k_linked_to_patients.csv'


In [29]:
table_name = "PatientChronicDiseases"

if table_name in all_data:
    removed_table = all_data.pop(table_name)
    print(f"✅ Table '{table_name}' has been removed from all_data.")
else:
    print(f"❌ Table '{table_name}' not found in all_data.")

✅ Table 'PatientChronicDiseases' has been removed from all_data.


In [30]:
print("📊 Shape of Each Table in 'Care_Stat':\n")
print(f"{'Table Name':<20} {'Rows':<8} {'Columns':<8}")
print("-" * 40)

for table_name, df in all_data.items():
    rows, cols = df.shape
    print(f"{table_name:<20} {rows:<8} {cols:<8}")

📊 Shape of Each Table in 'Care_Stat':

Table Name           Rows     Columns 
----------------------------------------
Doctors              10000    14      
DoctorPhones         10000    2       
Departments          10000    9       
DoctorDepartment     10000    3       
Department_Equipment 10000    2       
Patients             10000    10      
PatientPhones        10000    2       
Visits               10000    3       
DoctorWorkplaces     10000    2       
Appointments         10000    5       
ChronicDiseases      10000    2       
Medical_Records      10000    8       
Payments             10000    10      


In [31]:

print("🔍 NULL DATA ANALYSIS FOR ALL TABLES\n")
print(f"{'Table':<20} {'Rows':<8} {'Null Count':<12} {'Null %':<10}")
print("-" * 50)

summary_list = []  

for table_name, df in all_data.items():
    total_rows = len(df)
    null_count = df.isnull().sum().sum()  
    null_percent = (null_count / (total_rows * df.shape[1])) * 100 if total_rows > 0 else 0

    summary_list.append({
        'Table': table_name,
        'Total Rows': total_rows,
        'Null Count': null_count,
        'Null %': f"{null_percent:.2f}%"
    })

    print(f"{table_name:<20} {total_rows:<8} {null_count:<12} {null_percent:<10.2f}%")

summary_df = pd.DataFrame(summary_list)

🔍 NULL DATA ANALYSIS FOR ALL TABLES

Table                Rows     Null Count   Null %    
--------------------------------------------------
Doctors              10000    0            0.00      %
DoctorPhones         10000    0            0.00      %
Departments          10000    0            0.00      %
DoctorDepartment     10000    0            0.00      %
Department_Equipment 10000    0            0.00      %
Patients             10000    0            0.00      %
PatientPhones        10000    0            0.00      %
Visits               10000    0            0.00      %
DoctorWorkplaces     10000    0            0.00      %
Appointments         10000    0            0.00      %
ChronicDiseases      10000    0            0.00      %
Medical_Records      10000    0            0.00      %
Payments             10000    8969         8.97      %


## payments_null_data


In [36]:

source_table = "Departments"
target_table = "Payments"
col_name = "department_id"

print(f"🔄 Filling '{col_name}' in '{target_table}' using values from '{source_table}'\n")

if source_table not in all_data:
    print(f"❌ Source table '{source_table}' not found in all_data.")
elif target_table not in all_data:
    print(f"❌ Target table '{target_table}' not found in all_data.")
else:
    if col_name not in all_data[source_table]:
        print(f"❌ Column '{col_name}' not found in '{source_table}'.")
    else:
        dept_ids = all_data[source_table][col_name].dropna().unique()
        
        if len(dept_ids) == 0:
            print(f"❌ No valid {col_name} values found in '{source_table}'.")
        else:
            print(f"✅ Found {len(dept_ids)} unique {col_name} values: {sorted(dept_ids)}")
            
            n_payments = len(all_data[target_table])
            
            random_dept_ids = np.random.choice(dept_ids, size=n_payments, replace=True)
            
            all_data[target_table][col_name] = random_dept_ids
            
            print(f"✅ Successfully filled '{col_name}' in '{target_table}' with random values.")
            print(f"   Total rows updated: {n_payments}")
            
            print(f"\n🔍 Sample from '{target_table}' after update:")
            print(all_data[target_table][[col_name]].head(10))

🔄 Filling 'department_id' in 'Payments' using values from 'Departments'

✅ Found 10000 unique department_id values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 1

In [37]:

print("🔍 NULL DATA ANALYSIS FOR ALL TABLES\n")
print(f"{'Table':<20} {'Rows':<8} {'Null Count':<12} {'Null %':<10}")
print("-" * 50)

summary_list = []  

for table_name, df in all_data.items():
    total_rows = len(df)
    null_count = df.isnull().sum().sum()  
    null_percent = (null_count / (total_rows * df.shape[1])) * 100 if total_rows > 0 else 0

    summary_list.append({
        'Table': table_name,
        'Total Rows': total_rows,
        'Null Count': null_count,
        'Null %': f"{null_percent:.2f}%"
    })

    print(f"{table_name:<20} {total_rows:<8} {null_count:<12} {null_percent:<10.2f}%")

summary_df = pd.DataFrame(summary_list)

🔍 NULL DATA ANALYSIS FOR ALL TABLES

Table                Rows     Null Count   Null %    
--------------------------------------------------
Doctors              10000    0            0.00      %
DoctorPhones         10000    0            0.00      %
Departments          10000    0            0.00      %
DoctorDepartment     10000    0            0.00      %
Department_Equipment 10000    0            0.00      %
Patients             10000    0            0.00      %
PatientPhones        10000    0            0.00      %
Visits               10000    0            0.00      %
DoctorWorkplaces     10000    0            0.00      %
Appointments         10000    0            0.00      %
ChronicDiseases      10000    0            0.00      %
Medical_Records      10000    0            0.00      %
Payments             10000    0            0.00      %


## duplicate

In [38]:

print("🔍 ANALYSIS: Duplicated Rows in Each Table\n")
print(f"{'Table':<20} {'Total Rows':<12} {'Duplicated':<12} {'% Duplicated':<14}")
print("-" * 50)

duplicates_summary = []

for table_name, df in all_data.items():
    total_rows = len(df)
    duplicated_rows = df.duplicated().sum()
    duplicate_percent = (duplicated_rows / total_rows) * 100 if total_rows > 0 else 0

    duplicates_summary.append({
        'Table': table_name,
        'Total Rows': total_rows,
        'Duplicated': duplicated_rows,
        'Duplicate %': f"{duplicate_percent:.2f}%"
    })

    print(f"{table_name:<20} {total_rows:<12} {duplicated_rows:<12} {duplicate_percent:<14.2f}%")

summary_df = pd.DataFrame(duplicates_summary)

🔍 ANALYSIS: Duplicated Rows in Each Table

Table                Total Rows   Duplicated   % Duplicated  
--------------------------------------------------
Doctors              10000        0            0.00          %
DoctorPhones         10000        0            0.00          %
Departments          10000        0            0.00          %
DoctorDepartment     10000        0            0.00          %
Department_Equipment 10000        0            0.00          %
Patients             10000        0            0.00          %
PatientPhones        10000        0            0.00          %
Visits               10000        0            0.00          %
DoctorWorkplaces     10000        0            0.00          %
Appointments         10000        0            0.00          %
ChronicDiseases      10000        333          3.33          %
Medical_Records      10000        0            0.00          %
Payments             10000        0            0.00          %


📊 STARTING COMPREHENSIVE ANALYSIS FOR ALL TABLES...


📌 ANALYZING TABLE: 'Doctors'
------------------------------------------------------------
🔢 Shape: 10000 rows × 14 columns

📋 Data Types:
  → object: 7 column(s)
  → int64: 5 column(s)
  → float64: 2 column(s)

🧩 Missing Values: 0 cells (0.00%)
🔁 Duplicated Rows: 0 (0.00%)

📈 Numeric Columns - Descriptive Statistics:
       doctor_id       age  graduation_year  hire_year  years_of_experience  rating_avg    salary
count   10000.00  10000.00         10000.00   10000.00             10000.00    10000.00  10000.00
mean    49877.98     61.59          2002.53    2023.01                20.81        3.00  51463.19
std     28735.36     22.58            13.30       1.41                11.52        1.16  28434.15
min        10.00     23.00          1980.00    2021.00                 1.00        1.00   2031.58
25%     24964.50     42.00          1991.00    2022.00                11.00        2.00  26964.44
50%     49492.00     62.00          200

In [43]:
doctors = all_data['Doctors']

fig1 = px.box(doctors, x='specialization', y='salary',
              title="Doctors' salaries by specialty",
              labels={'specialization': 'Specialization', 'salary': 'Salary'},
              color='specialization',
              hover_data=['first_name', 'last_name', 'rating_avg'])

fig1.show()

In [44]:
patients = all_data['Patients']

fig2 = px.histogram(patients, x='age', nbins=30, color='gender',
                    title="Distribution of patients' ages by gender",
                    labels={'age': 'age', 'count': 'count'},
                    color_discrete_map={'male': 'lightblue', 'female': 'pink'})

fig2.show()

In [45]:
payments = all_data['Payments'].copy()
payments['payment_date'] = pd.to_datetime(payments['payment_date'], errors='coerce')

monthly_rev = payments.resample('M', on='payment_date')['amount'].sum().reset_index()

fig3 = px.line(monthly_rev, x='payment_date', y='amount',
               title="Monthly income",
               labels={'payment_date': 'Month', 'amount': 'amount'},
               markers=True)

fig3.show()

C:\Users\aliha\AppData\Local\Temp\ipykernel_14732\662688722.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [46]:
visits = all_data['Visits'].copy()
visits['visit_date'] = pd.to_datetime(visits['visit_date'], errors='coerce')

daily_visits = visits.resample('D', on='visit_date').size().reset_index(name='count')

fig4 = px.bar(daily_visits, x='visit_date', y='count',
              title="Number of daily visits",
              labels={'visit_date': 'date', 'count': 'Number of visits'},
              color='count',
              color_continuous_scale='Blues')

fig4.show()

In [ ]:
if 'doctor_id' in payments.columns and 'doctor_id' in doctors.columns:
    payments_with_docs = payments.merge(doctors[['doctor_id', 'specialization']], on='doctor_id', how='left')
    
    rev_by_spec = payments_with_docs.groupby('specialization')['amount'].sum().reset_index()

    fig5 = px.pie(rev_by_spec, values='amount', names='specialization',
                  title="Revenue percentage by specialty",
                  hole=0.4)  
    
    fig5.show()

In [52]:
output_folder = r'D:\instant\data analysis final\final_data_analysis'

os.makedirs(output_folder, exist_ok=True)

print(f"📁 Output folder: {output_folder}")
print("🔄 Saving cleaned tables...\n")

📁 Output folder: D:\instant\data analysis final\final_data_analysis
🔄 Saving cleaned tables...



In [53]:
if 'all_data' not in globals():
    print("❌ Error: 'all_data' is not defined. Please load and process your data first.")
else:
    if not all_data:
        print("❌ Error: 'all_data' is empty. No tables to save.")
    else:
        for table_name, df in all_data.items():
            file_path = os.path.join(output_folder, f"{table_name}.csv")
            
            try:
                df.to_csv(file_path, index=False, encoding='utf-8')
                
                print(f"✅ Saved: '{table_name}'")
                print(f"   📄 Path: {file_path}")
                print(f"   🔢 {len(df)} rows, {df.shape[1]} columns\n")
                
            except Exception as e:
                print(f"❌ Failed to save '{table_name}': {e}\n")

        print("🎉 All tables have been successfully saved to:")
        print(f"   {output_folder}")

✅ Saved: 'Doctors'
   📄 Path: D:\instant\data analysis final\final_data_analysis\Doctors.csv
   🔢 10000 rows, 14 columns

✅ Saved: 'DoctorPhones'
   📄 Path: D:\instant\data analysis final\final_data_analysis\DoctorPhones.csv
   🔢 10000 rows, 2 columns

✅ Saved: 'Departments'
   📄 Path: D:\instant\data analysis final\final_data_analysis\Departments.csv
   🔢 10000 rows, 9 columns

✅ Saved: 'DoctorDepartment'
   📄 Path: D:\instant\data analysis final\final_data_analysis\DoctorDepartment.csv
   🔢 10000 rows, 3 columns

✅ Saved: 'Department_Equipment'
   📄 Path: D:\instant\data analysis final\final_data_analysis\Department_Equipment.csv
   🔢 10000 rows, 2 columns

✅ Saved: 'Patients'
   📄 Path: D:\instant\data analysis final\final_data_analysis\Patients.csv
   🔢 10000 rows, 10 columns

✅ Saved: 'PatientPhones'
   📄 Path: D:\instant\data analysis final\final_data_analysis\PatientPhones.csv
   🔢 10000 rows, 2 columns

✅ Saved: 'Visits'
   📄 Path: D:\instant\data analysis final\final_data_anal